In [2]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
import os

In [3]:
load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_ACCESS_TOKEN")

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", 
    task="text-generation",
    huggingfacehub_api_token=hf_token
)

model = ChatHuggingFace(llm=llm)

d:\Langchain_models\venv_genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt=PromptTemplate(
    template='Generate 5 interesting facts about {topic}',
    input_variables=['topic']
)
parser=StrOutputParser()
chain=prompt|model|parser
result=chain.invoke({'topic':'Paris'})
print(result)

 1. Paris is known as the "City of Light" (La Ville Lumière) because it was a center of education and ideas during the Age of Enlightenment and also because it was one of the first cities to adopt street lighting.

2. The Eiffel Tower, one of the most recognized structures in the world, was built as a temporary exhibit for the 1889 World's Fair to celebrate the 100th anniversary of the French Revolution. It was initially criticized by some of France's leading artists and intellectuals for its design, but it has since become a global cultural icon of France and one of the most recognizable structures in the world.

3. Paris is home to the Louvre Museum, the world's largest art museum and a historic monument in Paris. It is home to thousands of works of art, including the Mona Lisa and the Venus de Milo. The museum is housed in the Louvre Palace, which was a royal palace before Louis XIV moved his court to Versailles in 1682.

4. Paris has been the setting for many classic films, includi

In [5]:
pip install grandalf

Note: you may need to restart the kernel to use updated packages.


In [6]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
   +-----------------+     
   | ChatHuggingFace |     
   +-----------------+     
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


In [8]:
prompt1=PromptTemplate(
    template='generate a detailed report on {topic}',
    input_variables=['topic']
)

prompt2=PromptTemplate(
    template='generate a 5 pointer summary from the following text \n {text}',
    input_variables=['text']
)

model=ChatHuggingFace(llm=llm)
parser=StrOutputParser()

chain=prompt1|model|parser|prompt2|model|parser
result=chain.invoke({'topic':'26/11 attacks in Mumbai, India'})
print(result)


 1. The 26/11 attacks in Mumbai, India, were a series of coordinated attacks by the Pakistan-based militant organization Lashkar-e-Taiba (LeT), resulting in 166 deaths and hundreds of injuries.
2. The attacks took place at seven locations, including Chhatrapati Shivaji Maharaj Terminus railway station, Leopold Cafe, Cama and Albless Hospital, Nariman House, Taj Mahal Palace Hotel, Oberoi Trident Hotel, and Metro Adlabs cinema.
3. The international community condemned the attacks, and Pakistan was accused of being behind them. The Indian government launched an investigation, leading to the identification and sentencing to death of one militant, while nine others were killed.
4. The attacks exposed weaknesses in India's security infrastructure and led to calls for greater investment in national security and intelligence capabilities.
5. Several memorials have been established in Mumbai to honor the victims, and the Taj Mahal Palace Hotel has been restored, symbolizing Mumbai's resilience

## Making a Parallel Chain

In [34]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import os

# Load .env
load_dotenv()
groq_token = os.getenv("GROQ_API_KEY")

# Set base URL for Groq (OpenAI-compatible)
groq_base_url = "https://api.groq.com/openai/v1"

# Model 1: Mixtral-8x7B (via Groq)
model1 = ChatOpenAI(
    openai_api_key=groq_token,
    openai_api_base=groq_base_url,
    model_name="meta-llama/llama-4-scout-17b-16e-instruct"
)

# Model 2: Gemma-7B-IT (via Groq)
model2 = ChatOpenAI(
    openai_api_key=groq_token,
    openai_api_base=groq_base_url,
    model_name="gemma2-9b-it"
)

In [36]:
from langchain.schema.runnable import RunnableParallel
from langchain_core.output_parsers import StrOutputParser


prompt1=PromptTemplate(
    template='generate short and simple notes from the {paper} research paper',
    input_variables=['paper']
)

prompt2=PromptTemplate(
    template='generate 5 short question answers from the following {paper}',
    input_variables=['paper']
)
prompt3=PromptTemplate(
    template='Merge the provided notes and quiz into a single document /n notes-> {notes} and quiz->{quiz}',
    input_variables=['notes','quiz']
)
parser=StrOutputParser()
parallel_chain=RunnableParallel({
    'notes':prompt1|model1|parser,
    'quiz':prompt2|model2|parser
})
merge_chain=prompt3|model1|parser
chain=parallel_chain|merge_chain
chain.invoke({'paper':'Attention is All you need'})

'## Attention is All You Need: A Brief Overview and Quiz\n\n### Introduction\n\nThe "Attention is All You Need" research paper introduced a new neural network architecture called Transformer. This architecture replaces traditional recurrent neural networks (RNNs) and convolutional neural networks (CNNs) with self-attention mechanisms, making it particularly suited for sequence-to-sequence tasks such as machine translation.\n\n### Key Components\n\n- **Self-Attention Mechanism**: This mechanism allows the model to attend to different parts of the input sequence simultaneously and weigh their importance. It takes in a set of query (Q), key (K), and value (V) vectors, computes attention weights by taking the dot product of Q and K and applying a softmax function, and uses these weights to compute a weighted sum of V.\n- **Encoder-Decoder Architecture**: The Transformer consists of an encoder and a decoder, both composed of identical layers. The encoder takes in a sequence of tokens and ou

In [37]:
chain.get_graph().print_ascii()

            +---------------------------+            
            | Parallel<notes,quiz>Input |            
            +---------------------------+            
                 **               **                 
              ***                   ***              
            **                         **            
+----------------+                +----------------+ 
| PromptTemplate |                | PromptTemplate | 
+----------------+                +----------------+ 
          *                               *          
          *                               *          
          *                               *          
  +------------+                    +------------+   
  | ChatOpenAI |                    | ChatOpenAI |   
  +------------+                    +------------+   
          *                               *          
          *                               *          
          *                               *          
+-----------------+         

## conditional chain

In [44]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import Field, BaseModel
from typing import Literal
from langchain.schema.runnable import RunnableBranch, RunnableLambda

parser=StrOutputParser()
class Feedback(BaseModel):
    sentiment: Literal['positive','negative','neutral']=Field(description='Give the sentiment of the feedback')

parser2=PydanticOutputParser(pydantic_object=Feedback)

prompt1=PromptTemplate(
    template="classify the sentiment of the following feeback text into positive or negetive or neutral \n {feedback} \n {format_instruction}",
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)
classifier_chain=prompt1|model2|parser2


prompt2=PromptTemplate(
    template="write an appropriate response to this positive feedback \n {feedback}",
    input_varibles=['feedback']
)

prompt3=PromptTemplate(
    template='write an appropriate response to this negative feedback \n{feedback}',
    input_variables=['feedback']

) 
prompt4=PromptTemplate(
    template="write an appropriate response to this neutral feedback \n {feedback}",
    input_varibles=['feedback']
)


branch_chain=RunnableBranch(
    (lambda x:x.sentiment=='positive',prompt2|model2|parser),
    (lambda x:x.sentiment=='negative',prompt3|model2|parser),
    (lambda x:x.sentiment=='neutral',prompt4|model2|parser),
    RunnableLambda(lambda x:"could not find sentiment")
)

chain=classifier_chain|branch_chain
print(chain.invoke({'feedback':"this is a fabulous phone from samsung. The camera is amazing, and the noise cancellation experience on calls is a very good feature to have. ALso I liked its pricing"}))

Here are some appropriate responses to positive feedback, depending on the context:

**Formal:**

* "Thank you for your kind words. I appreciate your feedback."
* "We are delighted to hear that you are satisfied with our [product/service]. Your feedback is valuable to us."
* "Thank you for taking the time to share your positive experience. We will continue to strive for excellence."

**Informal:**

* "Thanks so much! I'm really glad you liked it."
* "That's awesome to hear, thanks!"
* "I'm so happy you enjoyed it! Let me know if there's anything else I can do."

**Adding a Personal Touch:**

You can make your response even more personal by:

* **Specifying what you're grateful for:** "Thank you for your kind words about my presentation. I'm especially glad you found the [mention specific part] helpful."
* **Acknowledging the effort:** "I really appreciate your feedback. It means a lot to me knowing that my work is valued."
* **Offering to do more:** "Thank you for your positive feedbac